### 00 OpenAI API KEY 환경변수 등록

In [1]:
import os, json

with open('conf.json', 'r') as f:
        json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['openai_config']["API_KEY"]

### 01 설정

In [2]:
class CONFIG:
    etf_tkr="AIQ"

In [3]:
from data.get_data import get_pdf_df

pdf_df = get_pdf_df(etf_tkr=CONFIG.etf_tkr)

tkr_list = pdf_df.child_stk_tkr.to_list()

### 02 영향도 상위 5개 기업 추출

#### 02-1 2분기 가격데이터 로드

In [4]:
from data.get_data import get_prices_df
from data.fetch_data import fetch_data_from_db

prices_df = get_prices_df(tickers=tkr_list, start_date="2023-03-01")

#### 02-2 유동주식수 로드

In [5]:
import pandas as pd

sql = """
    SELECT * FROM os_stk_info;
"""

stk_info_df = pd.DataFrame(fetch_data_from_db(query=sql)).dropna()

stk_info_df = stk_info_df[stk_info_df["stk_tkr"].isin(prices_df.columns.str.upper().to_list())].reset_index(drop=True)

stk_info_df.index = stk_info_df['stk_tkr'].str.lower()

stk_info_df = stk_info_df[['float_shares']]

#### 02-3 영향도 상위 5개 기업 산출

In [6]:
시작_영향도 = stk_info_df['float_shares'] * prices_df.iloc[0]

끝_영향도 = stk_info_df['float_shares'] * prices_df.iloc[-1]

영향도 = abs(끝_영향도 - 시작_영향도)

영향도_top5_종목들_리스트 = 영향도.sort_values(ascending=False).head().index.str.upper().to_list()

영향도_top5_종목들_리스트

['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN']

### 03 뉴스데이터 로드

In [7]:
import yfinance as yf
from tqdm.auto import tqdm

stk_infos = yf.Tickers(영향도_top5_종목들_리스트)

display(tkr_list[0])

text = {
    f"{CONFIG.etf_tkr} news": []
}

for tkr in tqdm(영향도_top5_종목들_리스트):
    
    tmp = {}
    tmp["stock"] = tkr
    tmp["news_titles"] = []
    
    news_infos = stk_infos.tickers[tkr].news
    
    for news in news_infos:
        tmp['news_titles'].append(news["title"])
        
    text[f"{CONFIG.etf_tkr} news"].append(tmp)
    
text = str(text)

print(text)

'AAPL'

  0%|          | 0/5 [00:00<?, ?it/s]

{'AIQ news': [{'stock': 'AAPL', 'news_titles': ['Asia Stocks Steady Ahead of Key US Inflation Data: Markets Wrap', 'Apple Set to Unveil Latest iPhone: 5 Things to Watch', 'Dow Jones Futures Fall: Tesla Stock Breaks Out; Key Inflation Reports, Apple iPhone 15 Event Next', 'Qualcomm Inks Deal With Apple to Supply iPhone Chips Through 2026', 'Apple’s iPhone launch is in less than 24 hours. Here’s what to expect as CEO Tim Cook tries to reverse three painful quarters of sales shrinkage', 'Time to Buy Apple or Qualcomm Stock for Their Renewed Partnership', 'Apple launch event, Google vs. DOJ: What to watch', 'Google Goes to Federal Court. The Future of Internet Search Is at Stake.']}, {'stock': 'MSFT', 'news_titles': ['Microsoft (MSFT) Outpaces Stock Market Gains: What You Should Know', 'More writers sue OpenAI for copyright infringement over AI training', "ChatGPT's development nearly cost this small Midwestern city its drinking water", 'Top Stock Reports for Microsoft, Novo Nordisk & Book

### 03 프롬프트 엔지니어링

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI


template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                f"""
                Summarize the common issues based on the news headlines of stocks which are best 5 influential components of {CONFIG.etf_tkr} in Korean.
                
                - Tone : Humble
                - Style : MECE, accurate
                - Reader level : college student
                - Length : within 500 characters
                - Perspective : Mckinsey&Company senior consultant
                - Format : markdown
                
                You MUST answer in Korean. 
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

answer = llm(template.format_messages(text=text)).content

print(answer)

AIQ의 가장 영향력 있는 구성 요소인 AAPL, MSFT, NVDA, GOOGL, AMZN에 대한 뉴스 헤드라인을 종합해보면 다음과 같은 공통 이슈들이 나타납니다:

1. 주식 시장 동향: 주식 시장의 상황과 관련한 뉴스 헤드라인이 많이 보입니다. 예를 들어 "Asia Stocks Steady Ahead of Key US Inflation Data: Markets Wrap"이나 "Microsoft (MSFT) Outpaces Stock Market Gains: What You Should Know"와 같은 뉴스가 있습니다.

2. 제품 발표: 주요 기업들이 새로운 제품을 발표하는 경우도 많이 있습니다. "Apple Set to Unveil Latest iPhone: 5 Things to Watch"나 "Xbox to Launch the Xbox Mastercard, Its First-Ever Credit Card in the US, Issued by Barclays" 같은 뉴스가 해당됩니다.

3. 파트너십 협약: 기업 간의 파트너십 협약에 관한 뉴스도 종종 보입니다. "Qualcomm Inks Deal With Apple to Supply iPhone Chips Through 2026"이나 "Nvidia, Tesla, Meta And More Reveal Rules For Agonizing Market"과 같은 뉴스가 해당됩니다.

4. 경쟁과 소송: 경쟁이나 소송과 관련한 뉴스도 있습니다. "Alphabet Inc. (GOOG) Gains But Lags Market: What You Should Know"이나 "More writers sue OpenAI for copyright infringement over AI training"과 같은 뉴스가 해당됩니다.

5. 성장 전망: 기업의 성장 전망과 관련한 뉴스도 종종 보입니다. "AI Looks Set for Long-Term Growth: ETFs in Focus"이나 "One Company Is Ex